In [9]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler

# Load and preprocess the training and testing data
train_data = pd.read_csv("dataset/bank-note/train.csv", header=None)
test_data = pd.read_csv("dataset/bank-note/test.csv", header=None)

train_data.iloc[:, -1] = train_data.iloc[:, -1].map({1: 1, 0: -1})
test_data.iloc[:, -1] = test_data.iloc[:, -1].map({1: 1, 0: -1})

X_train = train_data.iloc[:, :-1].values
y_train = train_data.iloc[:, -1].values

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)

# Function to compute the Gaussian kernel matrix
def gaussian_kernel_matrix(X, sigma):
    n_samples = X.shape[0]
    K = np.zeros((n_samples, n_samples))
    for i in range(n_samples):
        for j in range(n_samples):
            K[i, j] = np.exp(-np.sum((X[i, :] - X[j, :]) ** 2) / (2 * sigma ** 2))
    return K

# Function to train the dual SVM with a Gaussian kernel using SLSQP method
def train_dual_svm_gaussian_slsqp(X, y, C, sigma):
    K = gaussian_kernel_matrix(X, sigma)
    
    def objective(alpha):
        return 0.5 * np.dot(alpha, np.dot(K, alpha * y) * y) - np.sum(alpha)
    
    constraints = ({'type': 'eq', 'fun': lambda alpha: np.sum(alpha * y), 'jac': lambda alpha: y})
    bounds = [(0, C) for _ in range(X.shape[0])]
    
    result = minimize(fun=objective,
                      x0=np.zeros(X.shape[0]),
                      method='SLSQP',
                      bounds=bounds,
                      constraints=constraints,
                      options={'ftol': 1e-6, 'disp': False})
    
    alphas = result.x
    sv = (alphas > 1e-5)
    
    return sv

# Convert data to NumPy arrays
X_train_np = X_train
y_train_np = y_train

# Define different values for gamma and C
gamma_values = [0.01, 0.1, 0.5, 1, 5, 100]
C_values = [100 / 873, 500 / 873, 700 / 873]

# Count the number of support vectors for each setting of gamma and C
print("Results for counting support vectors using SLSQP method:")
for C in C_values:
    for gamma in gamma_values:
        sv = train_dual_svm_gaussian_slsqp(X_train_np, y_train_np, C, gamma)
        num_sv = np.sum(sv)
        print(f"For C={C:.4f} and gamma={gamma:.2f}: Number of Support Vectors -> {num_sv}")
    print("-------------------------------------------------")

# Report the number of overlapped support vectors
print("\nOverlapped support vectors between consecutive gamma values for C=500/873 using SLSQP method:")
for i in range(len(gamma_values) - 1):
    gamma1 = gamma_values[i]
    gamma2 = gamma_values[i + 1]
    
    sv1 = train_dual_svm_gaussian_slsqp(X_train_np, y_train_np, C_values[1], gamma1)
    sv2 = train_dual_svm_gaussian_slsqp(X_train_np, y_train_np, C_values[1], gamma2)
    
    overlapped_sv_indices = np.where(np.logical_and(sv1, sv2))[0]
    num_overlapped_sv = len(overlapped_sv_indices)
    
    print(f"Gamma {gamma1:.2f} to {gamma2:.2f}: Overlapped Support Vectors -> {num_overlapped_sv}")


Results for counting support vectors using SLSQP method:
For C=0.1145 and gamma=0.01: Number of Support Vectors -> 872
For C=0.1145 and gamma=0.10: Number of Support Vectors -> 854


C:\Users\u1464674\AppData\Local\anaconda3\Lib\site-packages\scipy\optimize\_optimize.py:353: RuntimeWarning: Values in x were outside bounds during a minimize step, clipping to bounds
  warnings.warn("Values in x were outside bounds during a "


For C=0.1145 and gamma=0.50: Number of Support Vectors -> 425
For C=0.1145 and gamma=1.00: Number of Support Vectors -> 267
For C=0.1145 and gamma=5.00: Number of Support Vectors -> 558
For C=0.1145 and gamma=100.00: Number of Support Vectors -> 784
-------------------------------------------------
For C=0.5727 and gamma=0.01: Number of Support Vectors -> 872
For C=0.5727 and gamma=0.10: Number of Support Vectors -> 847
For C=0.5727 and gamma=0.50: Number of Support Vectors -> 194
For C=0.5727 and gamma=1.00: Number of Support Vectors -> 107
For C=0.5727 and gamma=5.00: Number of Support Vectors -> 259
For C=0.5727 and gamma=100.00: Number of Support Vectors -> 781
-------------------------------------------------
For C=0.8018 and gamma=0.01: Number of Support Vectors -> 872
For C=0.8018 and gamma=0.10: Number of Support Vectors -> 826
For C=0.8018 and gamma=0.50: Number of Support Vectors -> 183
For C=0.8018 and gamma=1.00: Number of Support Vectors -> 91
For C=0.8018 and gamma=5.00: 